In [ ]:
import numpy as np
import tensorflow
import matplotlib.pyplot as plt

In [ ]:
train_dir = '/content/drive/My Drive/Colab Notebooks/dataset/train'
validation_dir = '/content/drive/My Drive/Colab Notebooks/dataset/validation'
test_dir = '/content/drive/My Drive/Colab Notebooks/dataset/test'
nTrain = 480
nVal = 120
nTest = 150

In [ ]:
from tensorflow.keras.applications import VGG16
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
vgg_conv.summary()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rescale=1./255)
batchSize = 32

In [ ]:
t_features = np.zeros(shape=(nTrain, 7, 7, 512))
t_labels = np.zeros(shape=(nTrain, 3))
train_generator = datagen.flow_from_directory(train_dir, target_size=(224, 224), batch_size=batchSize, class_mode='categorical', shuffle=True)

In [ ]:
id = 0
print('The shape of train images: {}'.format(train_generator[id][0][0].shape))
plt.imshow(train_generator[id][0][0])
plt.axis('off')
train_image_label_id = np.argmax(train_generator[id][1][0])
classes_list = list(train_generator.class_indices.keys())
plt.title('Class name: {}'.format(classes_list[train_image_label_id]))
plt.show()

In [ ]:
i = 0
for inputs_batch, labels_batch in train_generator:
    features_batch = vgg_conv.predict(inputs_batch)
    t_features[i * batchSize : (i + 1) * batchSize] = features_batch
    t_labels[i * batchSize : (i + 1) * batchSize] = labels_batch
    i += 1
    if i * batchSize >= nTrain:
        break      
tra_feature_vectors = np.reshape(t_features, (nTrain, 7 * 7 * 512))
np.save('/content/drive/My Drive/Colab Notebooks/tra_features.npy', tra_feature_vectors)
np.save('/content/drive/My Drive/Colab Notebooks/tra_labels.npy', t_labels)

In [ ]:
v_features = np.zeros(shape=(nVal, 7, 7, 512))
v_labels = np.zeros(shape=(nVal, 3))
validation_generator = datagen.flow_from_directory(validation_dir, target_size=(224, 224), batch_size=batchSize, class_mode='categorical', shuffle=False)

In [ ]:
id = 0
print('The shape of validation images: {}'.format(validation_generator[id][0][0].shape))
plt.imshow(validation_generator[id][0][0])
plt.axis('off')
val_image_label_id = np.argmax(validation_generator[id][1][0])
classes_list = list(validation_generator.class_indices.keys())
plt.title('Class name: {}'.format(classes_list[val_image_label_id]))
plt.show()

In [ ]:
i = 0
for inputs_batch, labels_batch in validation_generator:
    features_batch = vgg_conv.predict(inputs_batch)
    v_features[i * batchSize : (i + 1) * batchSize] = features_batch
    v_labels[i * batchSize : (i + 1) * batchSize] = labels_batch
    i += 1
    if i * batchSize >= nVal:
        break
v_feature_vectors = np.reshape(v_features, (nVal, 7 * 7 * 512))
np.save('/content/drive/My Drive/Colab Notebooks/val_features.npy', v_feature_vectors)
np.save('/content/drive/My Drive/Colab Notebooks/val_labels.npy', v_labels)

In [ ]:
tst_features = np.zeros(shape=(nTest, 7, 7, 512))
tst_labels = np.zeros(shape=(nTest, 3))
test_generator = datagen.flow_from_directory(test_dir, target_size=(224, 224), batch_size=batchSize, class_mode='categorical', shuffle=False)

In [ ]:
id = 0
print('The shape of test images: {}'.format(test_generator[id][0][0].shape))
plt.imshow(test_generator[id][0][0])
plt.axis('off')
tes_image_label_id = np.argmax(test_generator[id][1][0])
classes_list = list(test_generator.class_indices.keys())
plt.title('Class name: {}'.format(classes_list[tes_image_label_id]))
plt.show()

In [ ]:
i = 0
for inputs_batch, labels_batch in test_generator:
    features_batch = vgg_conv.predict(inputs_batch)
    tst_features[i * batchSize : (i + 1) * batchSize] = features_batch
    tst_labels[i * batchSize : (i + 1) * batchSize] = labels_batch
    i += 1
    if i * batchSize >= nTest:
        break
tst_feature_vectors = np.reshape(tst_features, (nTest, 7 * 7 * 512))
np.save('/content/drive/My Drive/Colab Notebooks/tes_features.npy', tst_feature_vectors)
np.save('/content/drive/My Drive/Colab Notebooks/tes_labels.npy', tst_labels)

In [ ]:
train_features = np.load('/content/drive/My Drive/Colab Notebooks/tra_features.npy')
train_labels = np.load('/content/drive/My Drive/Colab Notebooks/tra_labels.npy')
validation_features = np.load('/content/drive/My Drive/Colab Notebooks/val_features.npy')
validation_labels = np.load('/content/drive/My Drive/Colab Notebooks/val_labels.npy')
test_features = np.load('/content/drive/My Drive/Colab Notebooks/tes_features.npy')
test_labels = np.load('/content/drive/My Drive/Colab Notebooks/tes_labels.npy')
print('Train Features: {}'.format(train_features.shape))
print('Validation Features: {}'.format(validation_features.shape))
print('Test Features: {}'.format(test_features.shape))

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
model = Sequential()
model.add(Dense(512, activation='relu', input_dim=7 * 7 * 512))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))
model.summary()

In [ ]:
from tensorflow.keras import optimizers
model.compile(optimizer=optimizers.RMSprop(learning_rate=2e-4), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_features, train_labels, epochs=20, batch_size=batchSize, validation_data=(validation_features, validation_labels))

In [ ]:
model_json = model.to_json()
with open('/content/drive/My Drive/Colab Notebooks/tl_model.json', 'w') as json_file:
    json_file.write(model_json)
model.save_weights('/content/drive/My Drive/Colab Notebooks/tl_model.h5')

In [ ]:
from tensorflow.keras.models import model_from_json
json_file = open('/content/drive/My Drive/Colab Notebooks/tl_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights('/content/drive/My Drive/Colab Notebooks/tl_model.h5')

In [ ]:
fnames = test_generator.filenames
ground_truth = test_generator.classes
label2index = test_generator.class_indices
idx2label = dict((v, k) for k, v in label2index.items())

In [ ]:
predictions = loaded_model.predict(test_features)
predicted_classes = np.argmax(predictions, axis=1)

In [ ]:
errors = np.where(predicted_classes != ground_truth)[0]
print('No of errors = {}/{}'.format(len(errors), nTest))

In [ ]:
from tensorflow.keras.preprocessing.image import load_img
for i in range(len(errors)):
    pred_class = np.argmax(predictions[errors[i]])
    pred_label = idx2label[pred_class]
    print('Original label: {}, Prediction: {}, Confidence: {:.3f}'.format(fnames[errors[i]].split('/')[0], pred_label, predictions[errors[i]][pred_class]))
    original = load_img('{}/{}'.format(test_dir, fnames[errors[i]]))
    plt.imshow(original)
    plt.show()

In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array
filename = '/content/drive/My Drive/Colab Notebooks/watermelon.jpg'
original = load_img(filename, target_size=(224, 224))
numpy_image = img_to_array(original)
image_batch = np.expand_dims(numpy_image, axis=0)
plt.imshow(np.uint8(image_batch[0]))
plt.show()
test_feature = vgg_conv.predict(image_batch)
test_feature = np.reshape(test_feature, (1, 7 * 7 * 512))
prediction = np.argmax(loaded_model.predict(test_feature), axis=1)
probability = loaded_model.predict(test_feature)
predicted_class = np.argmax(probability[0])
predict_label = idx2label[predicted_class]
print('Prediction: {}'.format(predict_label))